<a href="https://colab.research.google.com/github/raziehAskarizade/chatbot/blob/main/PA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labraries

In [ ]:
# impoert libararies
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, BatchNormalization, Conv2DTranspose, Concatenate, Cropping2D, ZeroPadding2D
from keras.optimizers import Adam
from sklearn.model_selection import ParameterGrid
import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Model
import tensorflow as tf
import pandas as pd
from sklearn.metrics import confusion_matrix

# connect to one drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Unit ANN

In [ ]:
# Define the U-Net architecture
def unet(input_size=(256, 256, 1)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Decoder
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = conv4
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = conv3
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = conv2
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = conv1
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)

    output = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=output)


    return model

UNET from Segmentation-of-Teeth-in-Panoramic-X-ray-Image-Using-U-Net


In [ ]:
def UNET(input_shape=(512,512,1),last_activation='sigmoid'):
    inputs=Input(shape=input_shape)

    conv1 = Conv2D(32,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    d1=Dropout(0.1)(conv1)
    conv2 = Conv2D(32,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d1)
    b=BatchNormalization()(conv2)

    pool1 = MaxPooling2D(pool_size=(2, 2))(b)
    conv3 = Conv2D(64,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    d2=Dropout(0.2)(conv3)
    conv4 = Conv2D(64,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d2)
    b1=BatchNormalization()(conv4)

    pool2 = MaxPooling2D(pool_size=(2, 2))(b1)
    conv5 = Conv2D(128,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    d3=Dropout(0.3)(conv5)
    conv6 = Conv2D(128,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d3)
    b2=BatchNormalization()(conv6)

    pool3 = MaxPooling2D(pool_size=(2, 2))(b2)
    conv7 = Conv2D(256,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    d4=Dropout(0.4)(conv7)
    conv8 = Conv2D(256,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d4)
    b3=BatchNormalization()(conv8)

    pool4 = MaxPooling2D(pool_size=(2, 2))(b3)
    conv9 = Conv2D(512,(3,3),activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    d5=Dropout(0.5)(conv9)
    conv10 = Conv2D(512,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d5)
    b4=BatchNormalization()(conv10)


    conv11 = Conv2DTranspose(512,(4,4), activation = 'relu', padding = 'same', strides=(2,2),kernel_initializer = 'he_normal')(b4)
    x= concatenate([conv11,conv8])
    conv12 = Conv2D(256,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)
    d6=Dropout(0.4)(conv12)
    conv13 = Conv2D(256,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d6)
    b5=BatchNormalization()(conv13)


    conv14 = Conv2DTranspose(256,(4,4), activation = 'relu', padding = 'same', strides=(2,2),kernel_initializer = 'he_normal')(b5)
    x1=concatenate([conv14,conv6])
    conv15 = Conv2D(128,3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x1)
    d7=Dropout(0.3)(conv15)
    conv16 = Conv2D(128,3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d7)
    b6=BatchNormalization()(conv16)

    conv17 = Conv2DTranspose(128,(4,4), activation = 'relu', padding = 'same',strides=(2,2), kernel_initializer = 'he_normal')(b6)
    x2=concatenate([conv17,conv4])
    conv18 = Conv2D(64,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x2)
    d8=Dropout(0.2)(conv18)
    conv19 = Conv2D(64,(3,3) ,activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d8)
    b7=BatchNormalization()(conv19)

    conv20 = Conv2DTranspose(64,(4,4), activation = 'relu', padding = 'same',strides=(2,2), kernel_initializer = 'he_normal')(b7)
    x3=concatenate([conv20,conv2])
    conv21 = Conv2D(32,(3,3) ,activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x3)
    d9=Dropout(0.1)(conv21)
    conv22 = Conv2D(32,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d9)

    outputs = Conv2D(1,(1,1), activation = last_activation, padding = 'same', kernel_initializer = 'he_normal')(conv22)
    model2 = Model( inputs = inputs, outputs = outputs)

    return model2

https://github.com/lorisrossi/unet/blob/master/unet.py

In [ ]:
def blob(shape=(512,512,1)):
  # U-Net architecture

  # Contracting path
  model_in = Input(shape=shape)
  conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(model_in)
  conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
  down1 = MaxPooling2D((2, 2), strides=2)(conv1)

  conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(down1)
  conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
  down2 = MaxPooling2D((2, 2), strides=2)(conv2)

  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(down2)
  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
  down3 = MaxPooling2D((2, 2), strides=2)(conv3)

  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(down3)
  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
  down4 = MaxPooling2D((2, 2), strides=2)(conv4)

  conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(down4)
  conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)

  # Expansive path
  up1 = UpSampling2D((2, 2))(conv5)
  conv6 = Conv2D(512, (2, 2), activation='relu', padding='same')(up1)
  concat1 = concatenate([conv4, conv6], axis=3)
  conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(concat1)
  conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

  up2 = UpSampling2D((2, 2))(conv6)
  conv7 = Conv2D(256, (2, 2), activation='relu', padding='same')(up2)
  concat2 = concatenate([conv3, conv7], axis=3)
  conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat2)
  conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

  up3 = UpSampling2D((2, 2))(conv7)
  conv8 = Conv2D(128, (2, 2), activation='relu', padding='same')(up3)
  concat3 = concatenate([conv2, conv8], axis=3)
  conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat3)
  conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

  up4 = UpSampling2D((2, 2))(conv8)
  conv9 = Conv2D(64, (2, 2), activation='relu', padding='same')(up4)
  concat4 = concatenate([conv1, conv9], axis=3)
  conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat4)
  conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

  conv10 = Conv2D(1, (1, 1), activation='softmax', padding='same')(conv9)

  model = Model(model_in, conv10)
  print(conv10.shape)
  return model

FCN dinamic    

   different size for each traning set

In [ ]:
def fcn(input_size=(None, None, 1)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)

    # Decoder
    up5 = UpSampling2D(size=(2, 2))(drop4)
    up5 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up5)
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up6)
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up7)
    print(up7.shape)
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    # Output
    output = Conv2D(1, 1, activation='sigmoid')(conv7)

    # Model
    model = Model(inputs=inputs, outputs=output)

    return model

Train model

In [ ]:
def train_model(X_train, Y_train, X_test, Y_test, param_grid, input_size, verbose=1, best_model=None):

    # Create the model with the current set of hyperparameters
    # model = UNET(input_size)
    # model = fcn(input_size)
    model = blob(input_size)
    metrics = ['accuracy']
    model.compile(optimizer=keras.optimizers.Adam(lr=param_grid['lr']), loss='binary_crossentropy', metrics= metrics)
    # Save the mode architecture
    compiled_model = model.save('/content/drive/MyDrive/compiled')

    # Train the model with the current set of hyperparameters
    fitness = model.fit(X_train, Y_train, batch_size=param_grid['batch_size'], epochs=param_grid['epochs'], verbose=verbose, validation_data=(X_test, Y_test))
    trained_model = model.save('/content/drive/MyDrive/trained')

    # Evaluate the model on the test set
    results = model.evaluate(X_test, Y_test)

    # Create a DataFrame to store the results and save to a CSV file
    df = pd.DataFrame({'Metric': model.metrics_names, 'Value': results})
    df.to_csv('evaluation_results.csv', index=False)

    return fitness

In [ ]:
def evaluate_model(X_test, Y_test, threshold=0.5):
  # if model stopped training or inttrupt or s.th
  model = keras.models.load_model('/content/drive/MyDrive/trained')

  # Predict the labels for the test set
  Y_pred = model.predict(X_test)
  Y_pred2 = Y_pred

  Y_pred = (Y_pred > threshold).astype(int)
  Y_test = np.where(Y_test > threshold, 1, 0)

  # flatten the masks to 1D arrays
  Y_test_flat = Y_test.ravel()
  Y_pred_flat = Y_pred.ravel()

  # Compute Pixel Accuracy
  pa = np.sum(Y_test_flat == Y_pred_flat) / len(Y_test_flat)

  # Compute confusion matrix
  cm = confusion_matrix(Y_test_flat, Y_pred_flat)

  # Compute IoU for each class
  iou = np.diag(cm) / (np.sum(cm, axis=1) + np.sum(cm, axis=0) - np.diag(cm))
  # Compute mean IoU
  miou = np.mean(iou)

  # calculate precision, recall, and f1-score for each segmentation class
  # set average='weighted' to calculate weighted averages of the metrics
  precision = precision_score(Y_test_flat, Y_pred_flat, average='weighted', zero_division=0)
  recall = recall_score(Y_test_flat, Y_pred_flat, average='weighted', zero_division=0)
  f1 = f1_score(Y_test_flat, Y_pred_flat, average='weighted', zero_division=0)

  # Calculate the evaluation metrics
  # wrong way!!
  accuracy = accuracy_score(Y_test_flat, Y_pred_flat)

  # Return the evaluation metrics as a dictionary
  return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'Pixel_Accuracy' : pa, 'mIoU' : miou}, X_test, Y_pred2, Y_test

Image processing On raw dataset and labels ones

In [ ]:
# image processing on raw images
def imgp_raw(path, size, the1, the2, the3=127, the4=200):

  image = cv.imread(path)

  # Resize the image to the input size of the U-Net model
  image = cv.resize(image,(size,size))


  # convert to grayscale
  img_ = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
  img_np = np.asarray(img_)


  # remove small obj
  kernel = np.ones((3, 3), np.float32)
  img_nr = cv.morphologyEx(img_np, cv.MORPH_OPEN, kernel)

  # threshold
  img_th = cv.threshold(img_nr, the1, the2, cv.THRESH_TRUNC)[1]

  # normalize
  dcm_gray = np.zeros([512, 512], dtype=np.uint8)
  img_th = np.uint8(cv.normalize(img_th, dcm_gray, the3, the4, cv.NORM_INF))

  # Enhancement
  equalized = cv.equalizeHist(img_th)


  # smoothing
  gblur = cv.GaussianBlur(equalized, (3, 3), 0, 0, cv.BORDER_WRAP)
  median = cv.medianBlur(gblur, 5)

  # denoising
  dst = cv.fastNlMeansDenoising(median, None, 20, 7, 21)

  # Normalize the pixel values to [0, 1]
  imagee = dst.astype(np.float32) / 255.0

  # Add a channel dimension to the image
  imagf = np.expand_dims(imagee, axis=-1)


  return imagf

In [ ]:
# image processing on labeled images
def imgp_label(path,size, the1, the2):
  image = cv.imread(path)

  # Resize the image to the input size of the U-Net model
  # height, width = image.shape[:2]
  # if max(height, width) <= size:
  #   pass
  # if height > width:
  #   new_height = size
  #   new_width = int(width * (size / height))
  # else:
  #   new_width = size
  #   new_height = int(height * (size / width))
  # image = cv.resize(image, (new_height, new_height))

  image = cv.resize(image, (size, size))

  # remove small obj
  kernel = np.ones((3, 3), np.float32)
  img_nr = cv.morphologyEx(image, cv.MORPH_OPEN, kernel)

  # Convert the image to grayscale
  image = cv.cvtColor(img_nr, cv.COLOR_BGR2GRAY)

  # threshold
  img_ = cv.threshold(image, the1, the2, cv.THRESH_BINARY)[1]

  # smoothing
  gblur = cv.GaussianBlur(img_, (3, 3), 0, 0, cv.BORDER_WRAP)
  median = cv.medianBlur(gblur, 5)

  # denoising
  dst = cv.fastNlMeansDenoising(median, None, 20, 7, 21)

  # Normalize the pixel values to [0, 1]
  imagee = dst.astype(np.float32) / 255.0

  # Add a channel dimension to the image
  mask = np.expand_dims(imagee, axis=-1)

  return mask

.npy

In [ ]:
# convert images file to .npy
def convert_npy(path_raw, path_label):

  # Load the image filenames in the labeled directory
  labeled_filenames = [filename for filename in os.listdir(path_label)]
  raw_filenames = [filename for filename in os.listdir(path_raw)]

  # Split data into train and test sets
  X_train, X_test, Y_train, Y_test = train_test_split(raw_filenames, labeled_filenames, test_size=0.2, random_state=42)

  # Load and preprocess the training and test images and labels
  Y_train_labeled = np.array([imgp_label(os.path.join(path_label, filename), 460, 210, 255) for filename in Y_train])
  X_train_raw = np.array([imgp_raw(os.path.join(path_raw, filename), 460, 160, 255) for filename in X_train])

  Y_test_labeled = np.array([imgp_label(os.path.join(path_label, filename), 460, 210, 255) for filename in Y_test])
  X_test_raw = np.array([imgp_raw(os.path.join(path_raw, filename), 460, 160, 255) for filename in X_test])


  # Save the preprocessed datasets to numpy files
  path = '/content/drive/MyDrive/'
  np.save(path + 'Y_train.npy', Y_train_labeled)
  np.save(path + 'X_train.npy', X_train_raw)
  np.save(path + 'Y_test.npy', Y_test_labeled)
  np.save(path + 'X_test.npy', X_test_raw)

In [ ]:
# !wget http://192.168.1.109:8000/

All functions

In [ ]:
# train model
# convert_npy('/content/drive/MyDrive/raw','/content/drive/MyDrive/results')

# Load the dataset
path = '/content/drive/MyDrive/dataset/'
X_train = np.load(path + 'X_train.npy')
Y_train = np.load(path + 'Y_train.npy')
X_test = np.load(path + 'X_test.npy')
Y_test = np.load(path + 'Y_test.npy')


# Specify the hyperparameter
# It can be get a lists of values and take a for loop to tain model for all possible hyperparameter
param_grid = {
    'lr': 1e-4,
    'batch_size': 5,
    'epochs': 30 }

# Train the model with different hyperparameters and find the best combination
# train_model(X_train, Y_train, X_test, Y_test, param_grid, (512, 512, 1))


# Evaluate the performance of the trained model on the binary thresholded test set
metrics ,  X_test2, Y_pred2, Y_test2 = evaluate_model(X_test, Y_test)

print(metrics)

x = np.arange(X_test2[0].ravel().size)
# for i in range(200):

#   fig, axs = plt.subplots(1, 3, figsize=(10, 3))

#   axs[0].imshow(X_test2[i].squeeze())
#   axs[0].set_title('X_test2')

#   axs[1].imshow(Y_test2[i].squeeze())
#   axs[1].set_title('Y_test2')

#   axs[2].imshow(Y_pred2[i].squeeze())
#   axs[2].set_title('Predicted')

#   plt.savefig(os.path.join('/content/drive/MyDrive/plots', "plot{}.png".format(i)))

ResourceExhaustedError: ignored